En este notebook lo que haremos será ver patrones y relaciones que tienen las disintas variables entre ellas. Para ello lo primero que haremos ser leer los datos escritos en el notebook anterior.

Montamos drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos las distintas rutas de los archivos

In [ ]:
# Cambiar el directorio de trabajo
%cd '/content/drive/My Drive/tfg/tfg/'

# Mostrar el directorio actual para confirmar el cambio
directorio_actual = %pwd
print("Directorio actual:", directorio_actual)

[Errno 2] No such file or directory: '/content/drive/My Drive/tfg/tfg/'
/content
Directorio actual: /content


In [ ]:
import os
import pandas as pd
path_actual = os.getcwd()
subdirectorio = 'datas'
file_train = 'df_train.csv'
path_train = os.path.join(path_actual, subdirectorio, file_train)


Leemos y metemos en un dataFrame llamado `df_train` los datos que usaremos para train

In [ ]:
import pandas as pd

df_train = pd.read_csv(path_train, index_col=None)
df_train

FileNotFoundError: [Errno 2] No such file or directory: '/content/datas/df_train.csv'

Mostramos la matriz de correlacion en train

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar el tamaño de la figura
plt.figure(figsize=(10, 8))

matriz_correlacion = df_train.corr()

# Crear un heatmap para visualizar la matriz de correlación
sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)

# Mostrar el gráfico
plt.show()

Como podemos ver en la matriz de correlacion y en el mapa de calor las variables que mas relacion tienen en nuestro caso son las varialbes hitX con hitTime. En este caso, tenemos tanta porcentaje de correlación entre una variable y otra debido a que la distancia en el eje X que viene representado por la variable `hitX` se calcula a partir del tiempo que tarda en detectarse el hit. La forma de calcular la distancia o el tiempo es con la formula $distancia = \cfrac{distancia}{velocidad}$. Por tanto, en el clasificador lo que haremos será eliminar una variable. En nuestro caso, eliminaremos la variable hitTime.

Otra pareja de variables que tiene también un alto indice de correlación son las variables PDGcode y trueE. En nuestro, experimento no podremos usar la variable trueE para clasificar las particulas puesto que en la vida real esta variable no es conocida, nos servirá para evaluar si la selección es mejor a una u otra energía.

Todas las demás su relación es muy pequeña como en el caso de la variavle PGDcode y hitZ y casi nula como en el resto de relaciones entre variables.

En nuestro caso, en la clasificación que vamos a realizar la vamos hacer a partir de la variable PGCcode, es decir el codigo de particula primaria va a ser nuestra etiqueta, por ese motivo nos interesaría que la variable PGCcode tuviera una relación más alta con alguna de las demás variables.

Ahora mostraremos distintos histogramas para conocer las distribuciones de los datos en función de si son kaones o piones.

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Filtrar el dataframe por PDGcode
# df_211 = df_train[df_train['PDGcode'] == 211]
# df_321 = df_train[df_train['PDGcode'] == 321]

# # Variables a visualizar
# variables = ['hitX', 'hitY', 'hitZ', 'hitInteg']

# # Configurar el estilo de los gráficos
# sns.set(style="whitegrid")

# # Crear las visualizaciones
# for var in variables:
#     plt.figure(figsize=(10, 6))

#     # Calcular los percentiles 1 y 99 para establecer los límites del eje x
#     low_percentile = np.percentile(df_train[var], 1)
#     high_percentile = np.percentile(df_train[var], 99)

#     # Gráfico para PDGcode = 211
#     sns.histplot(df_211[var], color="blue", label="PDGcode 211", kde=True, stat="density", linewidth=0, bins=30)

#     # Gráfico para PDGcode = 321
#     sns.histplot(df_321[var], color="red", label="PDGcode 321", kde=True, stat="density", linewidth=0, bins=30)

#     plt.title(f'Distribución de {var} por PDGcode')
#     plt.xlabel(var)
#     plt.ylabel('Densidad')

#     # Ajustar los límites de los ejes dinámicamente
#     plt.xlim(low_percentile, high_percentile)

#     plt.legend()
#     plt.show()


A continuación comentaremos las distribuciones de los datos en los dataFrame de train. Para ello veremos cada variable por separado. Antes de esto comentaremos que el $PDGcode = 211$ indicaria un pion, en cambio $PDGcode=321$ indicaria un kaon.

> **hitX**: En esta variable pordemos ver como en la parte central, cuando el valor de la variable tiende más a 0, vemos como se concentra una mayor parte de piones, en cambio a medida que se va separando, vamos encontrando más kaones. Esto quiere decir que el hits de los piones se produce más cercano a donde fue lanzada esa particula, en cambio el de los kaones se produce más lejano al punto donde fueron lanzados.

> **hitY**: Para el eje Y pasa lo mismo que en el eje X. Lo que viene a decir lo mismo, los hits de los piones se producen más cercanos al punto donde se produjo el experimento, mientras que a medida que nos vamos separando se encuentran más hits de kaones.

> **hitZ**: En el eje Z tiene un comportamiento distinto. Si el hits se ha producido en el rango de $(-∞,+20)$ hay más probabilidad de que la particula que vamos a predecir sea un kaons y si el hits se produce en el rango $(+20, +∞)$ hay más probabilidad de que sea un pions.

> **hitInteg**: En el caso de esta variable vemos como antes de que la magnitud de la variable sea 500 hay una mayor concentración de piones y a partir de 500 se estabilizan la cantidad de piones y kaones a medida que se incrementa el valor de la variable.





Como hemos visto en el valor de las variables hemos detectado un cierto patron en las posiciones de los hits en los ejes de coordenadas X,Y y Z. Por tanto, para ver un ejemplo en el espacio, vamos a representar 300 piones y 300 kaones en el eje de coordenadas, para ver como nuestro modelo pudiera llegar aprender un patrón para poder clasificar ambas variables.

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # Tomar 50 muestras aleatorias de cada dataframe
# sample_211 = df_211[['hitX', 'hitY', 'hitZ']].sample(n=300, random_state=42)
# sample_321 = df_321[['hitX', 'hitY', 'hitZ']].sample(n=300, random_state=42)

# # Crear figura para la visualización 3D
# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(111, projection='3d')

# # Agregar los puntos de cada muestra al gráfico
# ax.scatter(sample_211['hitX'], sample_211['hitY'], sample_211['hitZ'], c='blue', marker='o', label='PDGcode 211')
# ax.scatter(sample_321['hitX'], sample_321['hitY'], sample_321['hitZ'], c='red', marker='^', label='PDGcode 321')

# # Etiquetas de los ejes
# ax.set_xlabel('hitX')
# ax.set_ylabel('hitY')
# ax.set_zlabel('hitZ')

# # Título y leyenda
# plt.title('Distribución de hits en el espacio para PDGcodes 211 y 321')
# ax.legend()

# # Mostrar gráfico
# plt.show()


Como podemos ver, las particulas de los piones tienden a estar más agrupadas en el espacio, mientras que las partículas de los kaones tienen un rango más amplio de variación en su distribución espacial.

In [ ]:
df_train.columns

De todas las columnas que se muestran en la celda anterior, solo nos quedaremos para la clasificación con `hitX`, `hitY`, `hitZ` y `hitInteg`. Por esa razón se eliminan el resto de celdas a continuación.

In [ ]:
df_train = df_train.drop(['trueE'],axis=1)


In [ ]:
df_train.columns

Para pasarle al clasificador los datos debemos de hacer por eventos, es decir, pasarle todos los hits de un evento y la etiqueta que corresponda.

In [ ]:
# Suponiendo que df es tu DataFrame
conteo_hits = df_train['eventID'].value_counts()

# Para mostrar los resultados
print(conteo_hits)

Como podemos ver, tenemos diversos numero de hits por eventos. Al clasificador debemos de pasarle lineas con el mismo numero de eventos. Por ese motivo, debemos de elegir un numero suficiente para representar el numero de hits que le pasaremos al clasificador, en nuestro caso ese numero vendrá representado por la variable $N$. En los casos en el que el evento tenga un número menor de hits que de $N$, rellenaremos los huecos con 0, y en el caso contrario, que el evento tenga mayor numero de hits que de $N$, dejaremos los hits donde el tiempo sea mayor, es decir, se hayan producido después. Habrá que jugar con el valor de la variable $N$ para que el modelo no entrene con datos obsoletos como puede ser valores con 0 y tampoco pierda información al respecto para la clasificación.


Para ello definiremos una función, que pasandole como argumentos un dataFrame, ya que puede servir para depurar los datos de train como los de validación o test, devuelva una matriz.

In [ ]:
import pandas as pd

# Suponiendo que df es tu DataFrame y ya está cargado

# Filtrando el DataFrame para las filas donde eventID == 1 y PDGcode == 321
filtro = (df_train['eventID'] == 3) & (df_train['PDGcode'] == 321)

# Contando el número de filas que cumplen con el filtro
numero_hits = df_train[filtro].shape[0]

# Mostrando el resultado
print("El número exacto de hits para eventID=1 con PDGcode=321 es:", numero_hits)



In [ ]:
import pandas as pd

# Suponiendo que df es tu DataFrame

# Filtrando el DataFrame para las filas donde PDGcode == 321
df_filtrado = df_train[df_train['PDGcode'] == 321]

# Obteniendo los identificadores únicos de 'eventID' en el DataFrame filtrado
identificadores_unicos = df_filtrado['eventID'].unique()

# Mostrando los resultados
print(identificadores_unicos)
print(type(identificadores_unicos))


In [ ]:
import numpy as np

def filtrado_datos_concatenados(df, N):
    # Filtrado inicial por PDGcode
    codes = df['PDGcode'].unique()
    kaones = []
    labels = []

    df_filtrado = df[df['PDGcode'].isin(codes)]

    # Identificar todos los eventIDs únicos
    unique_eventIDs = df_filtrado['eventID'].unique()

    # Iterar sobre cada eventID único
    for eventID in unique_eventIDs:
        # Filtrando el DataFrame por eventID
        df_evento = df_filtrado[df_filtrado['eventID'] == eventID]

        # Ordenando por hitTime en orden descendente
        df_evento_ordenado = df_evento.sort_values(by='hitTime', ascending=False)

        # Ajustando el tamaño de los hits a N
        num_hits = min(len(df_evento_ordenado), N)

        # Inicializando listas para cada tipo de dato
        hitX_values, hitY_values, hitZ_values, hitInteg_values = [], [], [], []

        # Recolectando los valores separadamente
        for i in range(num_hits):
            hitX_values.append(df_evento_ordenado.iloc[i]['hitX'])
            hitY_values.append(df_evento_ordenado.iloc[i]['hitY'])
            hitZ_values.append(df_evento_ordenado.iloc[i]['hitZ'])
            hitInteg_values.append(df_evento_ordenado.iloc[i]['hitInteg'])

        # Concatenando los valores recolectados
        hit_values = hitX_values + hitY_values + hitZ_values + hitInteg_values

        # Rellenando con ceros si hay menos hits que N
        if len(hit_values) < 4 * N:  # 4 tipos de datos * N hits
            hit_values.extend([0] * (4 * N - len(hit_values)))

        # Asegurarse de limitar a 4*N para considerar casos donde se excede
        kaones.append(hit_values[:4 * N])
        labels.append(df_evento.iloc[0]['PDGcode'])  # Tomar el PDGcode del primer hit como etiqueta

    return np.array(kaones), np.array(labels)

In [ ]:
# Numero de hits por eventos que habrá dentro de las muestras
N = 20

In [ ]:
matriz_train, etiquetas_train = filtrado_datos_concatenados(df_train, N)

Para ver que es correcta la funcion `filtado_datos_concatenados`, el numero de identificadores únicos tiene que ser igual al numero de filas de la matriz que nos devuelve la funcion `filtado_datos_concatenados`.

In [ ]:
print('Longitud de la matriz que nos devuelve:', len(matriz_train))
print('Numero de identificadores unicos en el dataset:', len(identificadores_unicos))
print('Numero de etiquetas:', len(etiquetas_train))

In [ ]:
print('El numero de columnas por fila debe ser:', N*4)
print('El numero de columnas de la matriz es:', len(matriz_train[0]))


In [ ]:
print('Mostramos la primera fila para ver si tienen coherencia los resultados: ')
print(matriz_train[0])

## Estudio de la distribución del número de hits por eventos

Como hemos visto el valor de la varaible $N$ en el estudio de nuestro problema es algo fundamental para poder entrenar con el mayor número de detalles, cuando el valor de la variable es mayor, pero a la vez si hacemos muy grande este valor, algunas columnas de nuestro dataset tendrán el valor de 0 en muchas de sus casillas.


A la vez, si intentamos reducir el valor de $N$ para evitar entrenar con valores nulos, podremos perder el detalle de las muestras.

Para ello en esta sección estudiaremos su distribución en el conjunto train para ver los mejores valores.

In [ ]:
import matplotlib.pyplot as plt

# Suponiendo que conteo_hits es una Serie de Pandas que contiene el conteo de hits por eventID
# Histograma
plt.figure(figsize=(10, 6))
plt.hist(conteo_hits, bins=50, color='skyblue', edgecolor='black')
plt.title('Histograma del número de hits por eventID')
plt.xlabel('Número de hits')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(conteo_hits, bins=10, color='skyblue', edgecolor='black')
plt.title('Histograma del número de hits por eventID, percentiles')
plt.xlabel('Número de hits')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(conteo_hits, vert=False)
plt.title('Diagrama de caja del número de hits por eventID')
plt.xlabel('Número de hits')
plt.grid(True)
plt.show()

In [ ]:
percentiles = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99]
descripcion = conteo_hits.describe(percentiles=percentiles)

print(descripcion)

>El histograma nos muestra una distribución que parece ser aprosimadamente normal con una ligera inclinación hacia la derecha, indicando que hay más eventos con un número menor de hits y menos eventos con un número muy alto de hits. El pico esta entorno a los 1000 hits.

>En el diagrama de caja vemos como una mediana cernana a los 1000 hits con algunos valores atipicos que se extienden hacia el mayor número de hits. La caja que representa el ranfo intercuartílico, es estrecha en comparación con la gama de datos que tenemos, lo que sugiere que la mayoria de los datos están concentrados alrededor de la mediana.

>En el estudio numerico vemos como los valores cercanos a la mediana representa un caso promedia tipico.

Como vemos sería aconsejable aproximar este valor de $N$ casos promedios donde se producen mayor número de hits. Una cifra bastante asequible para empezar el entrenamiento seria con un valor de N comprendido en el rango [500,1500]. A medida que realicemos distintas prueblas con los modelos de clasificación acotaremos mucho más este valor

## Desbalanceo de clases

En esta sección veremos si necesitamos tratar el desbalanceo de clases viendo el número de etiquetas que tenemos de kaones y de piones en train.

In [ ]:
# Ahora, graficamos el histograma
plt.figure(figsize=(8, 4))
# Establecemos los bins justo entre las categorías que queremos separar
bins = [210.5, 321.5]  # Esto crea un bin para 211 y otro para 321

# Dibuja el histograma
plt.hist(etiquetas_train, bins=bins, align='mid', color='orange', rwidth=0.8)
# Etiquetas para el eje x
plt.xticks([211, 321], ['Clase 211', 'Clase 321'])

# Títulos y etiquetas
plt.title('Histograma del desbalanceo de clases con etiquetas 321 y 211')
plt.xlabel('Etiqueta de Clase')
plt.ylabel('Frecuencia')

# Mostrar el histograma
plt.show()